In [1]:
def prepare_backtesting_data(data_path,interval_in_mins):
    """
    data_path        : Path of the data
    interval_in_mins : No of minutes between 9:15 and 11 AM
    
    returns          : A dataframe with interval high and interval low
    """
    
    
    df = pd.read_csv(data_path)
    #df.drop('Unnamed: 0',axis = 1, inplace = True)
    df['time']= df.date.str.slice(10,16)
    df['day'] = df.date.str.slice(0,10)
    df["date"] = pd.to_datetime(df["date"])

    temp = df.groupby('day').head(interval_in_mins)
    interval_high_low = temp.groupby('day', as_index=False).agg({"high":"max", "low":"min"})
    interval_high_low.columns = ['day','interval_high','interval_low']

    df = pd.merge(df, interval_high_low, on = 'day')
    
    return df

In [2]:
def interval_check(d1,start_time,end_time):
    '''
    d1         : one single day's data
    start_time : time from when we start monitoring the stock prices
    end_time   : exit time of the startegy
    
    returns    : d1                 - data for within the desired start and end time
                 interval_high      - day high between 9:15 AM to 11 AM
                 interval_low       - day low between 9:15 AM to 11 AM
                 interval_high_time - time when the interval_high was clocked
                 interval_low_time  - time when the interval_low was clocked
    '''


    interval_high = d1['interval_high'].unique()[0]
    interval_low = d1['interval_low'].unique()[0]


    interval_high_time = d1[d1['high']==interval_high].date.dt.time.values[0]
    interval_low_time = d1[d1['low']==interval_low].date.dt.time.values[0]
    
    #subsetting the data for post to interval time
    d1 = d1.set_index('date')
    d1 = d1.between_time(start_time,end_time)
    d1 = d1.reset_index()

    return d1,interval_high,interval_low,interval_high_time,interval_low_time

In [3]:
def trade_entry(d1,position,interval_high,interval_low,target_param,loss_param,entry_buffer):
    
    '''
    d1            : one single day's data within the desired start and end time
    interval_high : day high between 9:15 AM to 11 AM
    interval_low  : day low between 9:15 AM to 11 AM
    target_param  : target points from the entry price
    loss_param    : stop loss price from the entry price
    entry_buffer  : this is number just in case we want to enter after 'x' points of high break/low break

    returns       :   entry_price - price at which we make the entry as per our startegy
                      entry_index - row number in our dataframe(d1) at which we entered the trade
                      stop_loss   - stop loss price at which we need to exit the trade
                      entry_time  - time when the we entered th trade
                      target      - target price at which we need to exit the trade

    '''
    
    
    if position == 'Long':

        entry_price = interval_high + entry_buffer
        entry_index = d1[d1['interval_high_breach'] == True].index[0]
        target    = entry_price + target_param
        stop_loss = entry_price - loss_param
        entry_time = d1['date'].dt.time[entry_index]    


    elif position == 'Short':
        entry_price = interval_low - entry_buffer
        entry_index = d1[d1['interval_low_breach'] == True].index[0]
        target    = entry_price - target_param
        stop_loss = entry_price + loss_param
        entry_time = d1['date'].dt.time[entry_index]
        
    #if no position was entered at all    
    else:
        entry_price = np.nan
        entry_index = np.nan
        position = "NA"
        target    = np.nan
        stop_loss = np.nan
        entry_time = np.nan


    return entry_price,entry_index,stop_loss,entry_time,target

In [4]:
def trade_check(d1,interval_high,interval_low,target_param,loss_param,entry_buffer):
    '''
    d1            : one single day's data within the desired start and end time
    interval_high : day high between 9:15 AM to 11 AM
    interval_low  : day low between 9:15 AM to 11 AM
    target_param  : target points from the entry price
    loss_param    : stop loss price from the entry price
    entry_buffer  : this is number just in case we want to enter after 'x' points of high break/low break
    
    returns       : entry_price - price at which we make the entry as per our startegy
                    entry_index - row number in our dataframe(d1) at which we entered the trade
                    stop_loss   - stop loss price at which we need to exit the trade
                    entry_time  - time when the we entered th trade
                    target      - target price at which we need to exit the trade
                    position    - position type "Short" or "Long"
    
    '''


    d1['interval_high_breach'] = np.where(d1.high > interval_high + entry_buffer, True, False)
    d1['interval_low_breach'] = np.where(d1.low < interval_low - entry_buffer, True, False)
    
    
    position = ''
    
    if len(d1[d1['interval_low_breach'] == True]) & len(d1[d1['interval_high_breach'] == True]):
        
        #print("Need to Calculate")
        
        long_entry_index = d1[d1['interval_high_breach'] == True].index[0]
        short_entry_index = d1[d1['interval_low_breach'] == True].index[0]
        
        
        
        if long_entry_index < short_entry_index:
            
            position = 'Long'
        
        else:
            
            position = 'Short'
            
    elif len(d1[d1['interval_low_breach'] == True]):
        
        position = 'Short'
        
    elif len(d1[d1['interval_high_breach'] == True]):
        
        position = 'Long'
        
    else:
        
        position = 'NA'

    entry_price,entry_index,stop_loss,entry_time,target = trade_entry(d1,
                                                                    position,
                                                                    interval_high,
                                                                    interval_low,
                                                                    target_param,
                                                                    loss_param,
                                                                    entry_buffer)
    
    
    return entry_price,entry_index,stop_loss,entry_time,target, position

In [5]:
def trade_result(d1,entry_index,target,stop_loss,position):
    
    '''
    d1          : one single day's data within the desired start and end time
    entry_index : row number in our dataframe(d1) at which we entered the trade
    target      : target price at which we need to exit the trade
    stop_loss   : stop loss price at which we need to exit the trade
    position    : position type "Short" or "Long"
    
    returns     : result        - outcome of the trade
                  closing_price - price at which the we exited the trade
                  exit_time     - exit time of the trade
    '''
   
    closing_price = ''
    exit_time = ''

    if position!="NA":
    
        post_signal_data = d1.iloc[entry_index:,:][["high","low","close","time"]].reset_index(drop=True).copy()        
        
        #iterate and check if target hits or stop loss hits
        if position == "Long":
            for candle_high, candle_low, closing,time in zip(post_signal_data.high,
                                                             post_signal_data.low,
                                                             post_signal_data.close,
                                                             post_signal_data.time):
                if candle_high > target:
                    result = 'Profit'
                    #print('Profit')
                    closing_price = target
                    exit_time = time
                    break
                elif candle_low < stop_loss:
                    result = 'Loss'
                    #print('Loss')
                    closing_price = stop_loss
                    exit_time = time
                    break
                else:
                    result = "Started but not finished"
                    closing_price = closing
                    exit_time = time
                    
        else:
            for candle_high, candle_low, closing, time in zip(post_signal_data.high,
                                                              post_signal_data.low,
                                                              post_signal_data.close,
                                                              post_signal_data.time):
                if candle_low < target:
                    result = 'Profit'
                    closing_price = target
                    exit_time = time
                    #print('Profit')
                    break
                elif candle_high > stop_loss:
                    result = 'Loss'
                    closing_price = stop_loss
                    exit_time = time
                    #print('Loss')
                    break
                else:
                    result = "Started but not finished"
                    last_price = post_signal_data
                    closing_price = closing
                    exit_time = time
    else:
        result = "NA"
        closing_price = 'NA'
                
    return result,closing_price,exit_time

In [6]:
def print_report(rpt_df,target_param,loss_param,lot_size,adjustment_factor,brokerage):
    '''
    rpt_df            : dataframe which captures the detailed outcome of our startegy for each day
    target_param      : target points from the entry price
    loss_param        : stop loss price from the entry price 
    lot_size          : quantity of stocks for 1 lot
    adjustment_factor : a number to consider slippage
    brokerage         : amount charged for one single trade by Zerodha

    returns           : final report with profit and loss and net returns for each day

    '''

    
    rpt_df['net_points_gained'] = np.where((rpt_df['result']=='Started but not finished')&(rpt_df['position']=='Long'),
                                           rpt_df['closing_price'] - rpt_df['entry_price'],
                                  np.where((rpt_df['result']=='Started but not finished')&(rpt_df['position']=='Short'),
                                           rpt_df['entry_price'] - rpt_df['closing_price'],0)                               
                                            )

    rpt_df['net_points_gained'] = np.where(rpt_df['result']=='Profit',target_param,rpt_df['net_points_gained'])

    rpt_df['net_points_gained'] = np.where(rpt_df['result']=='Loss',-loss_param,rpt_df['net_points_gained'])

    total_trading_days = rpt_df.shape[0]
    total_signal = rpt_df[rpt_df['result']!='NA'].shape[0]

    rpt_df = rpt_df.drop(index = rpt_df[rpt_df['result']=='NA'].index)
    rpt_df['net_result'] = np.where(rpt_df['net_points_gained']>0,"Profit","Loss")
    rpt_df['net_pnl'] = rpt_df.net_points_gained*lot_size*adjustment_factor - brokerage

    net_pnl_ratio = rpt_df.net_result.value_counts()['Profit']/rpt_df.net_result.value_counts()['Loss']
    total_point_earned = rpt_df.net_points_gained.sum()
    net_pnl = rpt_df['net_pnl'].sum()


    print("Total Loss : ",rpt_df.net_result.value_counts()['Loss'])
    print("Total Profit : ",rpt_df.net_result.value_counts()['Profit'])

    print("Profit to Loss Ratio : ",net_pnl_ratio)

    print("Net Points Earned : ",total_point_earned)
    print("Money Earned : ",np.round(net_pnl,2))
    
    
    percent_non_completion = rpt_df[rpt_df['result']=='Started but not finished'].shape[0]/rpt_df.shape[0]
    
    percent_non_completion = np.round(percent_non_completion,2)
    
    print("Percent not completed : ",percent_non_completion)

    return percent_non_completion, rpt_df,total_point_earned, np.round(net_pnl,2), net_pnl_ratio

In [7]:
def generate_bactesting_report(df,start_time,end_time,target_param,loss_param,muhurat_trading_day):
    
    '''
    df                  : raw data
    start_time          : time from when we start monitoring the stock prices
    end_time            : exit time of the startegy
    target_param        : target points from the entry price
    loss_param          : stop loss price from the entry price
    muhurat_trading_day : this is a special day where the trading hours were not between 9:15 AM to 3:30 PM
    
    returns             : a dataframe which captures the detailed outcome of our startegy for each day
    '''


    Days = df.day.unique().tolist()

    if muhurat_trading_day:

        Days.remove(muhurat_trading_day)

    rpt = []

    for day in Days:

        #print('Date --------  :   ',day)
        #checking for one single day
        d1 = df[df["day"] == day]
        
        Date = d1.date.dt.strftime("%d %B, %Y")

        d1,interval_high,interval_low,interval_high_time,interval_low_time = interval_check(d1,start_time,end_time)

        entry_price,entry_index,stop_loss,entry_time,target,position = trade_check(d1,interval_high,interval_low,target_param,loss_param,entry_buffer)

        result,closing_price,exit_time = trade_result(d1,entry_index,target,stop_loss,position)

        rpt.append((day,
                    interval_high,
                    interval_high_time,
                    interval_low,
                    interval_low_time,
                    position,
                    entry_time,
                    exit_time,
                    result,
                    entry_price,
                    closing_price))

        rpt_df = pd.DataFrame(rpt,columns= ['day',
                        'interval_high',
                        'interval_high_time',
                        'interval_low',
                        'interval_low_time',
                        'position',
                        'entry_time',
                        'exit_time',
                        'result',
                        'entry_price',
                        'closing_price'])
        
    
    return rpt_df

In [8]:
def create_single_parameter_report(target_param,data_path,
                                   interval_in_mins, loss_param,
                                    start_time, end_time,
                                    lot_size,adjustment_factor,brokerage,muhurat_trading_day):
    '''
    target_param        : target points from the entry price
    data_path           : Path of the data
    interval_in_mins    : No of minutes between 9:15 and 11 AM
    loss_param          : stop loss price from the entry price    
    start_time          : time from when we start monitoring the stock prices
    end_time            : exit time of the startegy
    lot_size            : quantity of stocks for 1 lot
    adjustment_factor   : a number to consider slippage
    brokerage           : amount charged for one single trade by Zerodha
    muhurat_trading_day : this is a special day where the trading hours were not between 9:15 AM to 3:30 PM
    
    returns            : final report with profit and loss and net returns for each day

    '''
    
    df = prepare_backtesting_data(data_path,interval_in_mins)

    rpt_df = generate_bactesting_report(df,start_time,end_time,target_param,loss_param,muhurat_trading_day)

    percent_non_completion, final_rpt, points_gained, money_earned, net_pnl_ratio = print_report(rpt_df,
                                                                         target_param,loss_param,
                                                                         lot_size,
                                                                         adjustment_factor,brokerage)
    
    return final_rpt    


In [9]:
import pandas as pd
import numpy as np
import datetime as dt

In [10]:
brokerage = 40
lot_size = 25
adjustment_factor = 0.95 #This variable is to incorporate STT deductions


target_param = 100

loss_param = 175

entry_buffer = 0

interval_dict = {"10 AM" :  45,
                 "11 AM" :  105,
                 "12 PM" :  165,
                 "1 PM"  :  225,
                 "2 PM"  :  285}


interval_in_mins = interval_dict['11 AM']
start_time = '10:59' #change it as per interval selection
end_time = '15:00'

data_path = '/Users/sreemantakesh/Desktop/The Algotrading Page/Datasets/FUTURES/BANK NIFTY FUT 2021.csv'
muhurat_trading_day  = "04-11-2021"

### Single Parameter

In [11]:
final_rpt = create_single_parameter_report(target_param,data_path,
                                   interval_in_mins, loss_param,
                                    start_time, end_time,
                                    lot_size,adjustment_factor,brokerage,muhurat_trading_day)

Total Loss :  71
Total Profit :  140
Profit to Loss Ratio :  1.971830985915493
Net Points Earned :  3365.150000000027
Money Earned :  71482.31
Percent not completed :  0.14


In [12]:
final_rpt

,day,interval_high,interval_high_time,interval_low,interval_low_time,position,entry_time,exit_time,result,entry_price,closing_price,net_points_gained,net_result,net_pnl
0,04-01-2021,31599.00,09:28:00,31200.00,10:57:00,Short,11:00:00,11:00,Profit,31200.00,31100.0,100,Profit,2335.0
1,05-01-2021,31296.20,10:54:00,31011.10,09:15:00,Long,11:05:00,11:07,Profit,31296.20,31396.2,100,Profit,2335.0
2,06-01-2021,31971.90,09:51:00,31798.25,09:15:00,Long,12:18:00,12:32,Profit,31971.90,32071.9,100,Profit,2335.0
3,07-01-2021,32229.95,09:34:00,32040.15,10:26:00,Long,12:30:00,15:00,Started but not finished,32229.95,32121.2,-108.75,Loss,-2622.8125
4,08-01-2021,32337.95,09:15:00,32131.00,09:42:00,Short,13:39:00,15:00,Started but not finished,32131.00,32210.0,-79.0,Loss,-1916.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,21-12-2021,35042.85,09:26:00,34672.10,09:39:00,Short,14:18:00,14:31,Profit,34672.10,34572.1,100,Profit,2335.0
238,22-12-2021,35060.00,09:20:00,34746.00,10:30:00,Long,14:59:00,15:00,Started but not finished,35060.00,35089.55,29.55,Profit,661.8125
239,23-12-2021,35410.00,09:15:00,35142.75,10:27:00,Long,12:17:00,13:18,Profit,35410.00,35510.0,100,Profit,2335.0
241,27-12-2021,34843.40,10:47:00,34264.35,09:21:00,Long,11:47:00,12:20,Profit,34843.40,34943.4,100,Profit,2335.0
